In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import csv
import gc

# Genre Dataset

## Import files

In [2]:
awards = pd.read_csv('./awards.csv')#, sep = ',', header = 0, names = ['eventId', 'eventName', 'awardName', 'year','occurence', 'winAnnouncementTime', 'categoryName', 'nomeneeNote', 'name'])
awards = awards.dropna(subset = ['eventName', 'awardName','isPrimary', 'isWinner', 'isTitle','isCompany', 'const'])

C:\Users\gonxh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies = pd.read_csv("data.tsv", sep='\t')
movies = movies.dropna(subset = ['tconst', 'originalTitle','startYear', 'genres'])

C:\Users\gonxh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merge datasets to create a genre/awards dataset

In [4]:
movies['genres'] = movies['genres'].apply(lambda x : x.split(','))


In [5]:
movies_merge = pd.merge(movies, awards, left_on='tconst', right_on='const',how='outer', indicator=False)

In [6]:
movies_merge = movies_merge.drop(columns = [
 'titleType',
 'primaryTitle',
 'isAdult',
 'endYear',
 'runtimeMinutes',
 'occurrence',
 'winAnnouncementTime',
 'categoryName',
 'nomeneeNote',
 'name',
 'originalName',
 'songNames',
 'episodeNames',
 'characterNames',
 'isWinner',
 'isPrimary',
 'isSecondary',
 'isPerson',
 'isTitle',
 'isCompany',
 'const',
 'notes'
])

In [7]:
awards_keys = list(awards.columns.values)

In [8]:
awards = awards.drop(columns = ['occurrence',
 'winAnnouncementTime',
 'categoryName',
 'nomeneeNote',
 'originalName',
 'songNames',
 'episodeNames',
 'characterNames',
 'isWinner',
 'isPrimary',
 'isSecondary',
 'isPerson',
 'isTitle',
 'isCompany',
 'notes'])

In [9]:
awards.rename(columns={'const':'id_movie'}, inplace=True)

In [10]:
movies_merge.rename(columns={'tconst':'id_movie'}, inplace=True)
movies_merge['year'] = movies_merge['year'].fillna(0.0).astype(int)
movies_merge = movies_merge.dropna(subset = ['id_movie', 'genres'])

Here we now have only the movies which are also in awards

In [11]:
splitting = movies_merge[['id_movie','genres']]

In [12]:
splitting=pd.DataFrame({'id_movie':splitting.id_movie.repeat(splitting.genres.str.len()),'genres':np.concatenate(splitting.genres.values)})

In [13]:
new = pd.merge(movies_merge, splitting, left_on='id_movie', right_on='id_movie',how='outer', indicator=False)


In [14]:
len(new)

103578622

Export in new file:

In [15]:
new.to_csv('merged_data.csv')